In [1]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

# custom imports
from multiprocessing import Pool        # Multiprocess Runs

warnings.filterwarnings('ignore')

In [2]:
pd.set_option('max_rows', 200)

In [3]:
########################### Helpers
#################################################################################
## Seeder
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [4]:
########################### Helper to load data by store ID
#################################################################################
# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    # Leave only relevant store
    df = df[df['store_id']==store]

    # With memory limits we have to read 
    # lags and mean encoding features
    # separately and drop items that we don't need.
    # As our Features Grids are aligned 
    # we can use index to keep only necessary rows
    # Alignment is good for us as concat uses less memory than merge.
    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2 # to not reach memory limit 
    
    df = pd.concat([df, df3], axis=1)
    del df3 # to not reach memory limit 
    
    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # Skipping first n rows
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test(base_path='.'):
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle(f'{base_path}/test_'+store_id+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [5]:
########################### Model params
#################################################################################
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'regression',
#                     'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': -1,
                } 

# Let's look closer on params

## 'boosting_type': 'gbdt'
# we have 'goss' option for faster training
# but it normally leads to underfit.
# Also there is good 'dart' mode
# but it takes forever to train
# and model performance depends 
# a lot on random factor 
# https://www.kaggle.com/c/home-credit-default-risk/discussion/60921

## 'objective': 'tweedie'
# Tweedie Gradient Boosting for Extremely
# Unbalanced Zero-inflated Data
# https://arxiv.org/pdf/1811.10192.pdf
# and many more articles about tweediie
#
# Strange (for me) but Tweedie is close in results
# to my own ugly loss.
# My advice here - make OWN LOSS function
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/140564
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/143070
# I think many of you already using it (after poisson kernel appeared) 
# (kagglers are very good with "params" testing and tuning).
# Try to figure out why Tweedie works.
# probably it will show you new features options
# or data transformation (Target transformation?).

## 'tweedie_variance_power': 1.1
# default = 1.5
# set this closer to 2 to shift towards a Gamma distribution
# set this closer to 1 to shift towards a Poisson distribution
# my CV shows 1.1 is optimal 
# but you can make your own choice

## 'metric': 'rmse'
# Doesn't mean anything to us
# as competition metric is different
# and we don't use early stoppings here.
# So rmse serves just for general 
# model performance overview.
# Also we use "fake" validation set
# (as it makes part of the training set)
# so even general rmse score doesn't mean anything))
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/133834

## 'subsample': 0.5
# Serves to fight with overfit
# this will randomly select part of data without resampling
# Chosen by CV (my CV can be wrong!)
# Next kernel will be about CV

##'subsample_freq': 1
# frequency for bagging
# default value - seems ok

## 'learning_rate': 0.03
# Chosen by CV
# Smaller - longer training
# but there is an option to stop 
# in "local minimum"
# Bigger - faster training
# but there is a chance to
# not find "global minimum" minimum

## 'num_leaves': 2**11-1
## 'min_data_in_leaf': 2**12-1
# Force model to use more features
# We need it to reduce "recursive"
# error impact.
# Also it leads to overfit
# that's why we use small 
# 'max_bin': 100

## l1, l2 regularizations
# https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c
# Good tiny explanation
# l2 can work with bigger num_leaves
# but my CV doesn't show boost
                    
## 'n_estimators': 1400
# CV shows that there should be
# different values for each state/store.
# Current value was chosen 
# for general purpose.
# As we don't use any early stopings
# careful to not overfit Public LB.

##'feature_fraction': 0.5
# LightGBM will randomly select 
# part of features on each iteration (tree).
# We have maaaany features
# and many of them are "duplicates"
# and many just "noise"
# good values here - 0.5-0.7 (by CV)

## 'boost_from_average': False
# There is some "problem"
# to code boost_from_average for 
# custom loss
# 'True' makes training faster
# BUT carefull use it
# https://github.com/microsoft/LightGBM/issues/1514
# not our case but good to know cons

In [9]:
########################### Vars
#################################################################################
VER = 1                          # Our model version
SEED = 42                        # We want all things
seed_everything(SEED)            # to be as deterministic 
lgb_params['seed'] = SEED        # as possible
N_CORES = psutil.cpu_count()     # Available CPU cores


#LIMITS and const
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1913               # End day of our train set
P_HORIZON   = 28                 # Prediction horizon
USE_AUX     = False               # Use or not pretrained models

#FEATURES to remove
## These features lead to overfit
## or values not present in test set
ROLLING_TEMP_FEATURES= ['rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_14_30', 'rolling_mean_tmp_14_7', 'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_60', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_14_60', 'rolling_mean_tmp_7_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_1_60']

remove_features = ['id','state_id','store_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

#PATHS for Features
ORIGINAL = '../input/m5-forecasting-accuracy/'
BASE     = '../cache/grid_part_1.pkl'
PRICE    = '../cache/grid_part_2.pkl'
CALENDAR = '../cache/grid_part_3.pkl'
LAGS     = '../cache/lags_df_28.pkl'
MEAN_ENC = '../cache/mean_encoding_df.pkl'


# AUX(pretrained) Models paths
AUX_MODELS = '../input/m5-aux-models/'


#STORES ids
STORES_IDS = pd.read_csv(ORIGINAL+'sales_train_validation.csv')['store_id']
STORES_IDS = list(STORES_IDS.unique())


#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])

In [10]:
########################### Aux Models
# If you don't want to wait hours and hours
# to have result you can train each store 
# in separate kernel and then just join result.

# If we want to use pretrained models we can 
## skip training 
## (in our case do dummy training
##  to show that we are good with memory
##  and you can safely use this (all kernel) code)
if USE_AUX:
    lgb_params['n_estimators'] = 2
    
# Here is some 'logs' that can compare
#Train CA_1
#[100]	valid_0's rmse: 2.02289
#[200]	valid_0's rmse: 2.0017
#[300]	valid_0's rmse: 1.99239
#[400]	valid_0's rmse: 1.98471
#[500]	valid_0's rmse: 1.97923
#[600]	valid_0's rmse: 1.97284
#[700]	valid_0's rmse: 1.96763
#[800]	valid_0's rmse: 1.9624
#[900]	valid_0's rmse: 1.95673
#[1000]	valid_0's rmse: 1.95201
#[1100]	valid_0's rmse: 1.9476
#[1200]	valid_0's rmse: 1.9434
#[1300]	valid_0's rmse: 1.9392
#[1400]	valid_0's rmse: 1.93446

#Train CA_2
#[100]	valid_0's rmse: 1.88949
#[200]	valid_0's rmse: 1.84767
#[300]	valid_0's rmse: 1.83653
#[400]	valid_0's rmse: 1.82909
#[500]	valid_0's rmse: 1.82265
#[600]	valid_0's rmse: 1.81725
#[700]	valid_0's rmse: 1.81252
#[800]	valid_0's rmse: 1.80736
#[900]	valid_0's rmse: 1.80242
#[1000]	valid_0's rmse: 1.79821
#[1100]	valid_0's rmse: 1.794
#[1200]	valid_0's rmse: 1.78973
#[1300]	valid_0's rmse: 1.78552
#[1400]	valid_0's rmse: 1.78158

In [ ]:
########################### Train Models
#################################################################################
if not USE_AUX:
    for store_id in STORES_IDS:
        print('Train', store_id)

        # Get grid for current store
        grid_df, features_columns = get_data_by_store(store_id)

        # Masks for 
        # Train (All data less than 1913)
        # "Validation" (Last 28 days - not real validatio set)
        # Test (All data greater than 1913 day, 
        #       with some gap for recursive features)
        train_mask = grid_df['d']<=END_TRAIN
        valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
        preds_mask = grid_df['d']>(END_TRAIN-100)

        # Apply masks and save lgb dataset as bin
        # to reduce memory spikes during dtype convertations
        # https://github.com/Microsoft/LightGBM/issues/1032
        # "To avoid any conversions, you should always use np.float32"
        # or save to bin before start training
        # https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
        train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                           label=grid_df[train_mask][TARGET])
        train_data.save_binary('train_data.bin')
        train_data = lgb.Dataset('train_data.bin')

        valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                           label=grid_df[valid_mask][TARGET])

        # Saving part of the dataset for later predictions
        # Removing features that we need to calculate recursively 
        grid_df = grid_df[preds_mask].reset_index(drop=True)
        keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
        grid_df = grid_df[keep_cols]
        grid_df.to_pickle('test_'+store_id+'.pkl')
        del grid_df

        # Launch seeder again to make lgb training 100% deterministic
        # with each "code line" np.random "evolves" 
        # so we need (may want) to "reset" it
        seed_everything(SEED)
        estimator = lgb.train(lgb_params,
                              train_data,
                              valid_sets = [valid_data],
                              verbose_eval = 100,
                              )

        # Save model - it's not real '.bin' but a pickle file
        # estimator = lgb.Booster(model_file='model.txt')
        # can only predict with the best iteration (or the saving iteration)
        # pickle.dump gives us more flexibility
        # like estimator.predict(TEST, num_iteration=100)
        # num_iteration - number of iteration want to predict with, 
        # NULL or <= 0 means use best iteration
        model_name = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin'
        pickle.dump(estimator, open(model_name, 'wb'))

        # Remove temporary files and objects 
        # to free some hdd space and ram memory
        !rm train_data.bin
        del train_data, valid_data, estimator
        gc.collect()

        # "Keep" models features for predictions
        MODEL_FEATURES = features_columns
else:
    _, MODEL_FEATURES = get_data_by_store(STORES_IDS[0])

Train CA_1
[100]	valid_0's rmse: 1.00773
[200]	valid_0's rmse: 0.97082
[300]	valid_0's rmse: 0.971714
[400]	valid_0's rmse: 0.966552
[500]	valid_0's rmse: 0.963775
[600]	valid_0's rmse: 0.958889
[700]	valid_0's rmse: 0.953146
[800]	valid_0's rmse: 0.950001
[900]	valid_0's rmse: 0.944664
[1000]	valid_0's rmse: 0.939991
[1100]	valid_0's rmse: 0.935821
[1200]	valid_0's rmse: 0.931327
[1300]	valid_0's rmse: 0.927571
[1400]	valid_0's rmse: 0.923566
Train CA_2
[100]	valid_0's rmse: 1.0185
[200]	valid_0's rmse: 0.940232
[300]	valid_0's rmse: 0.920841
[400]	valid_0's rmse: 0.911309
[500]	valid_0's rmse: 0.904589
[600]	valid_0's rmse: 0.898855
[700]	valid_0's rmse: 0.893969
[800]	valid_0's rmse: 0.88978
[900]	valid_0's rmse: 0.885548
[1000]	valid_0's rmse: 0.881871
[1100]	valid_0's rmse: 0.878435
[1200]	valid_0's rmse: 0.875188
[1300]	valid_0's rmse: 0.871836
[1400]	valid_0's rmse: 0.869163
Train CA_3
[100]	valid_0's rmse: 1.41164
[200]	valid_0's rmse: 1.38128
[300]	valid_0's rmse: 1.43118
[400

In [ ]:
MODEL_FEATURES

In [73]:
########################### Predict
#################################################################################

# Create Dummy DataFrame to store predictions
all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

# Timer to measure predictions time 
main_time = time.time()

# Loop over each prediction day
# As rolling lags are the most timeconsuming
# we will calculate it for whole day
for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # Make temporary grid to calculate rolling lags
    grid_df = base_test.copy()
    grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)
        
    for store_id in STORES_IDS:
        
        # Read all our models and make predictions
        # for each day/store pairs
        model_path = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin' 
        USE_AUX = False
        if USE_AUX:
            model_path = AUX_MODELS + model_path
        
        estimator = pickle.load(open(model_path, 'rb'))
        
        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        store_mask = base_test['store_id']==store_id
        
        mask = (day_mask)&(store_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])
    
    # Make good column naming and add 
    # to all_preds DataFrame
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds

Predict | Day: 1
##########  0.42 min round |  0.42 min total |  36916.81 day sales |
Predict | Day: 2
##########  0.40 min round |  0.83 min total |  35039.39 day sales |
Predict | Day: 3
##########  0.40 min round |  1.23 min total |  34496.99 day sales |
Predict | Day: 4
##########  0.41 min round |  1.63 min total |  34949.53 day sales |
Predict | Day: 5
##########  0.40 min round |  2.04 min total |  40687.38 day sales |
Predict | Day: 6
##########  0.40 min round |  2.44 min total |  49878.18 day sales |
Predict | Day: 7
##########  0.39 min round |  2.84 min total |  53270.81 day sales |
Predict | Day: 8
##########  0.40 min round |  3.23 min total |  44298.27 day sales |
Predict | Day: 9
##########  0.40 min round |  3.63 min total |  44061.54 day sales |
Predict | Day: 10
##########  0.40 min round |  4.04 min total |  39255.66 day sales |
Predict | Day: 11
##########  0.40 min round |  4.44 min total |  40844.48 day sales |
Predict | Day: 12
##########  0.40 min round |  4.84

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.848568,0.742027,0.759141,0.834251,0.998605,1.111911,1.275494,0.920254,0.999363,...,0.892533,1.174632,1.142005,0.880456,0.827838,0.896932,0.858370,0.924446,1.146147,1.137668
1,HOBBIES_1_002_CA_1_validation,0.179878,0.162982,0.147953,0.174403,0.224758,0.281586,0.340958,0.225941,0.214544,...,0.228661,0.262662,0.283927,0.201004,0.181838,0.179628,0.205281,0.198598,0.267046,0.283098
2,HOBBIES_1_003_CA_1_validation,0.533609,0.552040,0.594672,0.543559,0.664334,0.726649,0.789302,0.666366,0.658130,...,0.655079,0.806398,0.823376,0.612408,0.561612,0.562896,0.601879,0.728195,0.792919,0.834710
3,HOBBIES_1_004_CA_1_validation,1.832088,1.393147,1.302757,1.484324,1.950798,2.790336,2.934027,1.797238,1.417257,...,1.831508,2.398252,2.790351,1.812308,1.502331,1.302116,1.294284,1.927021,2.932778,2.983924
4,HOBBIES_1_005_CA_1_validation,0.997682,0.909933,0.862777,0.896248,1.037838,1.376248,1.533417,1.095121,0.987339,...,1.017622,1.428382,1.471818,1.033954,0.867569,0.889845,0.896009,1.116724,1.432921,1.582681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0.370195,0.326864,0.301811,0.289316,0.382721,0.407593,0.413526,0.443670,0.471238,...,0.449130,0.562555,0.636631,0.481847,0.384576,0.344609,0.316375,0.414396,0.451798,0.520949
30486,FOODS_3_824_WI_3_validation,0.293401,0.236331,0.249674,0.239736,0.284877,0.353075,0.329982,0.364915,0.357170,...,0.319993,0.393618,0.430626,0.295627,0.259125,0.259098,0.235032,0.242141,0.292981,0.277371
30487,FOODS_3_825_WI_3_validation,0.671438,0.532639,0.514616,0.548111,0.664500,0.769764,0.930050,1.065146,1.022740,...,1.054324,1.404313,1.484300,1.088239,0.801698,0.769520,0.739453,0.816523,0.934643,0.971624
30488,FOODS_3_826_WI_3_validation,0.815190,0.749725,0.659933,0.662442,0.701988,1.025856,1.036325,1.162237,1.096226,...,0.942206,1.227796,1.304982,0.849048,0.772911,0.665723,0.729238,0.882862,0.924427,1.013000


In [59]:
########################### Predict
#################################################################################

# Create Dummy DataFrame to store predictions
all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

# Timer to measure predictions time 
main_time = time.time()

# Loop over each prediction day
# As rolling lags are the most timeconsuming
# we will calculate it for whole day
for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # Make temporary grid to calculate rolling lags
    grid_df = base_test.copy()
#     grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)
        
    for store_id in [STORES_IDS[0]]:
        
        # Read all our models and make predictions
        # for each day/store pairs
        model_path = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin' 
        if USE_AUX:
            model_path = AUX_MODELS + model_path
        
        estimator = pickle.load(open(model_path, 'rb'))
        
        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        store_mask = base_test['store_id']==store_id
        
        mask = (day_mask)&(store_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])
    
    # Make good column naming and add 
    # to all_preds DataFrame
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds

Predict | Day: 1
##########  0.01 min round |  0.01 min total |  4049.89 day sales |
Predict | Day: 2
##########  0.01 min round |  0.02 min total |  3568.27 day sales |
Predict | Day: 3
##########  0.01 min round |  0.03 min total |  3496.02 day sales |
Predict | Day: 4
##########  0.01 min round |  0.04 min total |  3580.76 day sales |
Predict | Day: 5
##########  0.01 min round |  0.05 min total |  4308.12 day sales |
Predict | Day: 6
##########  0.01 min round |  0.06 min total |  5768.03 day sales |
Predict | Day: 7
##########  0.01 min round |  0.07 min total |  6117.64 day sales |
Predict | Day: 8
##########  0.01 min round |  0.08 min total |  4439.99 day sales |
Predict | Day: 9
##########  0.01 min round |  0.10 min total |  4177.12 day sales |
Predict | Day: 10
##########  0.01 min round |  0.11 min total |  3930.75 day sales |
Predict | Day: 11
##########  0.01 min round |  0.12 min total |  3869.88 day sales |
Predict | Day: 12
##########  0.01 min round |  0.13 min total 

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.772000,0.734914,0.734893,0.720987,0.840034,0.956533,1.145050,0.729010,0.813038,...,0.847025,1.117781,1.016531,0.815601,0.801748,0.754584,0.741047,0.880228,1.009224,1.027774
1,HOBBIES_1_002_CA_1_validation,0.201083,0.186337,0.189955,0.204204,0.233496,0.337094,0.371614,0.215739,0.201566,...,0.172945,0.210950,0.217041,0.152504,0.134901,0.201116,0.214553,0.225449,0.301759,0.282394
2,HOBBIES_1_003_CA_1_validation,0.399354,0.382745,0.402756,0.415152,0.579339,0.665032,0.618992,0.369373,0.360845,...,0.470862,0.569594,0.602274,0.436734,0.397831,0.425472,0.455423,0.539800,0.626101,0.632120
3,HOBBIES_1_004_CA_1_validation,1.805859,1.321388,1.289095,1.541088,2.007748,2.934195,3.201199,1.531729,1.348519,...,1.804394,2.646150,3.041863,1.581248,1.302862,1.192595,1.246205,1.781065,2.929898,3.209640
4,HOBBIES_1_005_CA_1_validation,0.883101,0.769470,0.933032,0.837224,1.090231,1.437012,1.506958,0.992105,0.948832,...,0.986478,1.416491,1.522765,0.956606,0.857248,0.832595,0.857353,1.044839,1.447442,1.398809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30486,FOODS_3_824_WI_3_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30487,FOODS_3_825_WI_3_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30488,FOODS_3_826_WI_3_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
########################### Export
#################################################################################
# Reading competition sample submission and
# merging our predictions
# As we have predictions only for "_validation" data
# we need to do fillna() for "_evaluation" items
submission = pd.read_csv(ORIGINAL+'sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv('submission_v'+str(VER)+'.csv', index=False)

In [10]:
# Summary

# Of course here is no magic at all.
# No "Novel" features and no brilliant ideas.
# We just carefully joined all
# our previous fe work and created a model.

# Also!
# In my opinion this strategy is a "dead end".
# Overfits a lot LB and with 1 final submission 
# you have no option to risk.


# Improvement should come from:
# Loss function
# Data representation
# Stable CV
# Good features reduction strategy
# Predictions stabilization with NN
# Trend prediction
# Real zero sales detection/classification


# Good kernels references 
## (the order is random and the list is not complete):
# https://www.kaggle.com/ragnar123/simple-lgbm-groupkfold-cv
# https://www.kaggle.com/jpmiller/grouping-items-by-stockout-pattern
# https://www.kaggle.com/headsortails/back-to-predict-the-future-interactive-m5-eda
# https://www.kaggle.com/sibmike/m5-out-of-stock-feature
# https://www.kaggle.com/mayer79/m5-forecast-attack-of-the-data-table
# https://www.kaggle.com/yassinealouini/seq2seq
# https://www.kaggle.com/kailex/m5-forecaster-v2
# https://www.kaggle.com/aerdem4/m5-lofo-importance-on-gpu-via-rapids-xgboost


# Features were created in these kernels:
## 
# Mean encodings and PCA options
# https://www.kaggle.com/kyakovlev/m5-custom-features
##
# Lags and rolling lags
# https://www.kaggle.com/kyakovlev/m5-lags-features
##
# Base Grid and base features (calendar/price/etc)
# https://www.kaggle.com/kyakovlev/m5-simple-fe


# Personal request
# Please don't upvote any ensemble and copypaste kernels
## The worst case is ensemble without any analyse.
## The best choice - just ignore it.
## I would like to see more kernels with interesting and original approaches.
## Don't feed copypasters with upvotes.

## It doesn't mean that you should not fork and improve others kernels
## but I would like to see params and code tuning based on some CV and analyse
## and not only on LB probing.
## Small changes could be shared in comments and authors can improve their kernel.

## Feel free to criticize this kernel as my knowlege is very limited
## and I can be wrong in code and descriptions. 
## Thank you.